<a href="https://colab.research.google.com/github/pampam01/deteksi-kata-toxic/blob/main/deteksi_toxic_word_untuk_berbagai_bahasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U datasets transformers accelerate evaluate scikit-learn torch emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 41.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2


In [2]:
import re
import random
import numpy as np
import torch
import emoji
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer
)
import evaluate
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

In [3]:

# -----------------------
# Config
# -----------------------
MODEL_NAME = "xlm-roberta-base"
DATASET_NAME = "FredZhang7/toxi-text-3M"
TEXT_COL = "text"
LABEL_COL = "is_toxic"
LANG_COL = "lang"

SEED = 42
MAX_LENGTH = 256
BATCH_SIZE = 16
LR = 2e-5
NUM_EPOCHS = 2
FP16 = True
WEIGHTED_LOSS = True

# For quick test (set to None for full dataset)
MAX_TRAIN_SAMPLES = 50_000
MAX_EVAL_SAMPLES  = 10_000

In [4]:
# -----------------------
# Reproducibility
# -----------------------
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(SEED)

In [5]:
# -----------------------
# Load dataset
# -----------------------
raw_train = load_dataset(DATASET_NAME, split="train")
raw_val   = load_dataset(DATASET_NAME, split="validation")
raw_test  = load_dataset(DATASET_NAME, split="test")

def subsample(ds, n):
    if n is None or n >= len(ds):
        return ds
    idx = np.random.default_rng(SEED).choice(len(ds), size=n, replace=False)
    return ds.select(sorted(idx))

train_ds = subsample(raw_train, MAX_TRAIN_SAMPLES)
val_ds   = subsample(raw_val,   MAX_EVAL_SAMPLES)
test_ds  = subsample(raw_test,  MAX_EVAL_SAMPLES)

print(train_ds, val_ds, test_ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train/multilingual-train-deduplicated.cs(…):   0%|          | 0.00/1.45G [00:00<?, ?B/s]

multilingual-validation%28new%29.csv: 0.00B [00:00, ?B/s]

test/multilingual-test.csv:   0%|          | 0.00/28.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2880667 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29641 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/63812 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'is_toxic', 'lang'],
    num_rows: 50000
}) Dataset({
    features: ['text', 'is_toxic', 'lang'],
    num_rows: 10000
}) Dataset({
    features: ['text', 'is_toxic', 'lang'],
    num_rows: 10000
})


In [6]:
# -----------------------
# Data Cleaning
# -----------------------
def clean_text(text):
    # Lowercase
    text = text.lower()
    # Replace URLs, mentions, hashtags
    text = re.sub(r"http\S+|www.\S+", " URL ", text)
    text = re.sub(r"@\w+", " USER ", text)
    text = re.sub(r"#\w+", " HASHTAG ", text)
    # Remove emoji (optional: could also keep them)
    text = emoji.replace_emoji(text, replace=" ")
    # Remove non-alphanumeric except basic punctuation
    text = re.sub(r"[^a-zA-Z0-9\s\.,!?]", " ", text)
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_clean(batch):
    batch[TEXT_COL] = [clean_text(t) for t in batch[TEXT_COL]]
    return batch

train_ds = train_ds.map(preprocess_clean, batched=True)
val_ds   = val_ds.map(preprocess_clean, batched=True)
test_ds  = test_ds.map(preprocess_clean, batched=True)



Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
train_ds

Dataset({
    features: ['text', 'is_toxic', 'lang'],
    num_rows: 50000
})

In [8]:
val_ds[0]

{'text': 'you fuck your dad', 'is_toxic': 1, 'lang': 'es'}

In [9]:
test_ds[0]

{'text': 'doctor who adl viki ba l na 12. doctor olarak bir viki yazar kendi ad n eklemi tir. ahsen d zelttim. onaylarsan z sevinirim. occipital',
 'is_toxic': 0,
 'lang': 'tr'}

In [10]:
# -----------------------
# Tokenization
# -----------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tokenize_batch(example):
    return tokenizer(
        example[TEXT_COL],
        truncation=True,
        max_length=MAX_LENGTH
    )

tokenized = DatasetDict({
    "train": train_ds.map(tokenize_batch, batched=True, remove_columns=[TEXT_COL]),
    "validation": val_ds.map(tokenize_batch, batched=True, remove_columns=[TEXT_COL]),
    "test": test_ds.map(tokenize_batch, batched=True, remove_columns=[TEXT_COL]),
})


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [26]:
# -----------------------
# Compute class weights
# -----------------------
def compute_class_weights(dataset, label_col=LABEL_COL):
    labels = np.array(dataset[label_col])
    counts = np.bincount(labels, minlength=2)
    weights = len(labels) / (2.0 * counts + 1e-9)
    return torch.tensor(weights, dtype=torch.float)

class_weights = None
if WEIGHTED_LOSS:
    class_weights = compute_class_weights(train_ds, LABEL_COL)
    print("Class weights:", class_weights.tolist())

Class weights: [0.5841940641403198, 3.4693310260772705]


In [12]:
# -----------------------
# Model
# -----------------------
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# -----------------------
# Metrics
# -----------------------
metric_acc = evaluate.load("accuracy")
metric_f1  = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": metric_f1.compute(predictions=preds, references=labels, average="macro")["f1"],
        "f1_micro": metric_f1.compute(predictions=preds, references=labels, average="micro")["f1"],
    }


In [27]:
# -----------------------
# Custom Trainer (weighted loss)
# -----------------------
class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, **kwargs):
        super().__init__(**kwargs)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        if self.class_weights is not None:
            cw = self.class_weights.to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss(weight=cw)
        else:
            loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [28]:
# -----------------------
# Training
# -----------------------
training_args = TrainingArguments(
    output_dir="./toxixlmr",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    fp16=FP16,
    logging_steps=100,
    report_to="none",
    seed=SEED
)

In [29]:
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"].rename_column(LABEL_COL, "labels"),
    eval_dataset=tokenized["validation"].rename_column(LABEL_COL, "labels"),
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
    class_weights=class_weights
)


/tmp/ipython-input-1554600069.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(**kwargs)


In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro
1,0.371100,0.949064,0.653100,0.652760,0.653100
2,0.323600,0.936615,0.647100,0.647099,0.647100


TrainOutput(global_step=6250, training_loss=0.3964679187011719, metrics={'train_runtime': 1798.2513, 'train_samples_per_second': 55.61, 'train_steps_per_second': 3.476, 'total_flos': 1.198164990562944e+16, 'train_loss': 0.3964679187011719, 'epoch': 2.0})

In [31]:

# -----------------------
# Evaluation
# -----------------------
print("Validation metrics:", trainer.evaluate(tokenized["validation"].rename_column(LABEL_COL, "labels")))
print("Test metrics:", trainer.evaluate(tokenized["test"].rename_column(LABEL_COL, "labels")))

preds = trainer.predict(tokenized["test"].rename_column(LABEL_COL, "labels"))
y_true = preds.label_ids
y_pred = np.argmax(preds.predictions, axis=1)
print("\nClassification Report:\n", classification_report(y_true, y_pred, digits=4, target_names=["non_toxic","toxic"]))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))


Validation metrics: {'eval_loss': 0.9490635991096497, 'eval_accuracy': 0.6531, 'eval_f1_macro': 0.6527598122604734, 'eval_f1_micro': 0.6531, 'eval_runtime': 22.5007, 'eval_samples_per_second': 444.431, 'eval_steps_per_second': 27.777, 'epoch': 2.0}
Test metrics: {'eval_loss': 0.67555832862854, 'eval_accuracy': 0.7729, 'eval_f1_macro': 0.6805362197441529, 'eval_f1_micro': 0.7729, 'eval_runtime': 33.3965, 'eval_samples_per_second': 299.433, 'eval_steps_per_second': 18.715, 'epoch': 2.0}

Classification Report:
               precision    recall  f1-score   support

   non_toxic     0.8665    0.8386    0.8523      7814
       toxic     0.4826    0.5380    0.5088      2186

    accuracy                         0.7729     10000
   macro avg     0.6745    0.6883    0.6805     10000
weighted avg     0.7825    0.7729    0.7772     10000


Confusion Matrix:
 [[6553 1261]
 [1010 1176]]


In [33]:
#  -----------------------
# Per-language Evaluation
# -----------------------
def eval_per_language(raw_test_ds, tokenized_test_ds, max_langs=10):
    lang_counts = Counter(raw_test_ds[LANG_COL])
    top_langs = [l for l,_ in lang_counts.most_common(max_langs)]
    for lang in top_langs:
        idx = [i for i, l in enumerate(raw_test_ds[LANG_COL]) if l == lang]
        ds_tok = tokenized_test_ds.select(idx).rename_column(LABEL_COL, "labels")
        out = trainer.predict(ds_tok)
        y = out.label_ids
        p = np.argmax(out.predictions, axis=1)
        acc = (p == y).mean()
        f1 = metric_f1.compute(predictions=p, references=y, average="macro")["f1"]
        print(f"[{lang}] size={len(idx)} acc={acc:.3f} f1_macro={f1:.3f}")

eval_per_language(test_ds, tokenized["test"], max_langs=10)


[tr] size=2234 acc=0.899 f1_macro=0.647


[pt] size=1750 acc=0.853 f1_macro=0.718


[fr] size=1700 acc=0.750 f1_macro=0.713


[ru] size=1639 acc=0.527 f1_macro=0.505


[it] size=1331 acc=0.811 f1_macro=0.710


[es] size=1196 acc=0.765 f1_macro=0.707


[en] size=90 acc=0.644 f1_macro=0.640


[de] size=23 acc=0.522 f1_macro=0.487


[so] size=9 acc=0.778 f1_macro=0.679


[bg] size=4 acc=0.250 f1_macro=0.200


In [35]:
# -----------------------
# Inference helper
# -----------------------
def predict_texts(texts):
    model.eval()
    inputs = tokenizer(texts, truncation=True, padding=True, max_length=MAX_LENGTH, return_tensors="pt").to(model.device)
    with torch.no_grad():
        logits = model(**inputs).logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()
    preds = probs.argmax(axis=1)
    return [{"text": t, "pred": int(p), "p_toxic": float(probs[i][1])} for i,(t,p) in enumerate(zip(texts, preds))]

samples = [
    "I hope you have a great day!",
    "kamu itu goblok banget",
    "Tu es vraiment stupide",
    "あなたは素晴らしい友達です"
]
print("\nSample predictions:\n", predict_texts(samples))


Sample predictions:
 [{'text': 'I hope you have a great day!', 'pred': 0, 'p_toxic': 0.00854433048516512}, {'text': 'kamu itu goblok banget', 'pred': 1, 'p_toxic': 0.972388505935669}, {'text': 'Tu es vraiment stupide', 'pred': 1, 'p_toxic': 0.9841241240501404}, {'text': 'あなたは素晴らしい友達です', 'pred': 0, 'p_toxic': 0.051272422075271606}]
